In [1]:
# This changes the current directory to the base saga directory - make sure to run this first!
# This is necessary to be able to import the py files and use the right directories,
# while keeping all the notebooks in their own directory.
import os
import sys

if 'saga_base_dir' not in locals():
    saga_base_dir = os.path.abspath('..')
if saga_base_dir not in sys.path:
    os.chdir(saga_base_dir)

In [2]:
from glob import glob
import numpy as np

import erikutils

from astropy import units as u
from astropy import modeling
from astropy.table import Table
from astropy.io import fits
from astropy import visualization

In [3]:
%matplotlib inline
from matplotlib import rcParams, style
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

rcParams['image.cmap'] = 'viridis'
rcParams['image.origin'] = 'lower'
rcParams['axes.prop_cycle'] = style.library['seaborn-deep']['axes.prop_cycle']
rcParams['figure.figsize'] = (14, 8)
rcParams['axes.titlesize'] =  rcParams['axes.labelsize'] = 16
rcParams['xtick.labelsize'] =  rcParams['ytick.labelsize'] = 14

In [4]:
# valid for erik's computer
os.environ['DEEP_DIR'] = '/Volumes/ErikDroboData/data/DEIMOS/'

In [5]:
dirs = {nm: os.path.join(os.environ['DEEP_DIR'], nm) for nm in ('Gil1', 'DN1', 'MD2')}
spec1d_fns = {nm: glob(os.path.join(dirs[nm], 'spec1d*')) for nm in dirs}
slit_fns = {nm: glob(os.path.join(dirs[nm], 'slit*')) for nm in dirs}

In [ ]:
def show_all_slits(target, scalekwargs=dict(scale='asinh', percent=99)):
    plt.figure(figsize=(16, 24))

    for i, fn in enumerate(slit_fns[target]):
        plt.subplot(12, 2, i+1)
        erikutils.deimosutils.show_deimos_slit(fn, scalekwargs=scalekwargs)
        plt.title(os.path.split(fn)[-1])
    plt.tight_layout()

In [ ]:
def find_match(substr, target):
    idx = [i for i, fn in enumerate(fns[target]) if substr in fn]
    assert len(idx)==1
    return spec1d_fns[target][idx[0]]

# Gil1 

In [ ]:
show_all_slits('Gil1')

Must be the thing in 009

In [ ]:
good = []
for fn in fns['Gil1']:
    if '009' not in fn or 'serendip' in fn:
        continue
        
    plt.figure(figsize=(16,8))
    xb, bspec, xr, rspec = erikutils.deimosutils.plot_deimos_spec1d(fn, smoothing=-2)
    good.append((fn, xb, bspec, xr, rspec))
    plt.title(os.path.split(fn)[-1])

In [ ]:
xb, bspec, xr, rspec = good[0][1:]

msk = (5900<xb)&(xb<6100)

model0 = (modeling.models.Gaussian1D(amplitude=10000,mean=5965, stddev=1.5) + 
          modeling.models.Gaussian1D(amplitude=30000,mean=6025, stddev=2) + 
          modeling.models.Const1D(amplitude=1000))

model = modeling.fitting.LevMarLSQFitter()(model0, xb[msk], bspec[msk])

plt.figure(figsize=(16, 6))
plt.plot(xb[msk], bspec[msk])
plt.plot(xb[msk], model0(xb[msk]))
plt.plot(xb[msk], model(xb[msk]))
model

In [ ]:
np.array([model.mean_0.value/4959, model.mean_1.value/5007])-1

Target is clearly at a redshift of ~.2, so not a satellite

# DN1 

In [ ]:
show_all_slits('DN1')

# MD2 

In [ ]:
show_all_slits('MD2')